# SQLite Graph Database Extension Tutorial

This notebook demonstrates how to use the SQLite Graph Database Extension from Python. The extension provides graph database functionality with openCypher-like query support built on top of SQLite.

## Prerequisites

- Python 3.6+
- sqlite3 module (built-in)
- Jupyter notebook
- Compiled graph extension (.so file)

## Features

- **Graph storage**: Nodes and edges with JSON properties
- **Graph algorithms**: Shortest path, centrality, connectivity
- **Cypher queries**: openCypher-compatible query parsing
- **Write operations**: CREATE, MERGE, SET, DELETE
- **Performance**: Built on SQLite for reliability and speed

## 1. Setup and Extension Loading

In [ ]:
import sqlite3
import json
import os
from pathlib import Path

# Connect to SQLite database
conn = sqlite3.connect(":memory:")  # In-memory database
conn.row_factory = sqlite3.Row  # Enable column access by name
cursor = conn.cursor()

# Load the graph extension
extension_path = "../build/libgraph.so"  # Adjust path as needed

if os.path.exists(extension_path):
    conn.enable_load_extension(True)
    conn.load_extension(extension_path)
    print(f"✅ Graph extension loaded: {extension_path}")
else:
    print(f"❌ Extension not found: {extension_path}")
    print("Please make sure the extension is compiled and the path is correct.")

In [ ]:
# Test basic functionality
cursor.execute("SELECT graph_count_nodes() as node_count")
result = cursor.fetchone()
print(f"Initial node count: {result['node_count']}")
print("✅ Extension is working correctly!")

## 2. Creating Nodes and Edges

In [ ]:
# Create person nodes with properties
people = [
    (1, {"name": "Alice", "age": 30, "occupation": "Developer", "city": "New York"}),
    (2, {"name": "Bob", "age": 25, "occupation": "Designer", "city": "San Francisco"}),
    (3, {"name": "Carol", "age": 28, "occupation": "Manager", "city": "Chicago"}),
    (4, {"name": "Dave", "age": 32, "occupation": "Analyst", "city": "Austin"})
]

print("Creating person nodes...")
for person_id, properties in people:
    cursor.execute("SELECT graph_node_add(?, ?) as result", 
                   (person_id, json.dumps(properties)))
    result = cursor.fetchone()
    print(f"✅ {properties['name']} (ID: {person_id}): {result['result']}")

# Check node count
cursor.execute("SELECT graph_count_nodes() as count")
result = cursor.fetchone()
print(f"\nTotal nodes created: {result['count']}")

In [ ]:
# Create relationships between people
relationships = [
    (1, 2, "FRIENDS", {"since": "2020-01-15", "closeness": 0.8}),
    (1, 3, "WORKS_WITH", {"project": "GraphDB", "since": "2019-06-01"}),
    (2, 4, "FRIENDS", {"since": "2018-11-20", "closeness": 0.9}),
    (3, 4, "COLLABORATES", {"project": "Analytics", "frequency": "weekly"})
]

print("Creating relationships...")
for from_id, to_id, rel_type, properties in relationships:
    cursor.execute("SELECT graph_edge_add(?, ?, ?, ?) as result",
                   (from_id, to_id, rel_type, json.dumps(properties)))
    result = cursor.fetchone()
    
    # Get names for display
    from_name = next(p[1]['name'] for p in people if p[0] == from_id)
    to_name = next(p[1]['name'] for p in people if p[0] == to_id)
    
    print(f"✅ {from_name} --{rel_type}--> {to_name}: {result['result']}")

# Check edge count
cursor.execute("SELECT graph_count_edges() as count")
result = cursor.fetchone()
print(f"\nTotal edges created: {result['count']}")

## 3. Graph Statistics and Analysis

In [ ]:
# Get overall graph statistics
cursor.execute("SELECT graph_count_nodes() as nodes, graph_count_edges() as edges")
stats = cursor.fetchone()

print("📊 Graph Statistics:")
print(f"   Nodes: {stats['nodes']}")
print(f"   Edges: {stats['edges']}")

# Calculate graph properties
try:
    cursor.execute("SELECT graph_density() as density")
    result = cursor.fetchone()
    print(f"   Density: {result['density']:.3f}")
except sqlite3.Error as e:
    print(f"   Density: Error - {e}")

try:
    cursor.execute("SELECT graph_is_connected() as connected")
    result = cursor.fetchone()
    print(f"   Connected: {bool(result['connected'])}")
except sqlite3.Error as e:
    print(f"   Connected: Error - {e}")

try:
    cursor.execute("SELECT graph_has_cycle() as has_cycle")
    result = cursor.fetchone()
    print(f"   Has cycles: {bool(result['has_cycle'])}")
except sqlite3.Error as e:
    print(f"   Has cycles: Error - {e}")

## 4. Node Centrality Analysis

In [ ]:
# Calculate degree centrality for each person
print("🌟 Node Centrality Analysis:")
print("   (Higher values indicate more connected nodes)")
print()

for person_id, properties in people:
    try:
        cursor.execute("SELECT graph_degree_centrality(?) as centrality", (person_id,))
        result = cursor.fetchone()
        centrality = result['centrality']
        name = properties['name']
        print(f"   {name:8} (ID {person_id}): {centrality:.3f}")
    except sqlite3.Error as e:
        print(f"   {properties['name']}: Error - {e}")

## 5. Path Finding

In [ ]:
# Find shortest paths between people
print("🛤️  Shortest Path Analysis:")
print()

path_queries = [
    (1, 4, "Alice", "Dave"),
    (2, 3, "Bob", "Carol"),
    (1, 3, "Alice", "Carol")
]

for from_id, to_id, from_name, to_name in path_queries:
    try:
        cursor.execute("SELECT graph_shortest_path(?, ?) as path", (from_id, to_id))
        result = cursor.fetchone()
        path = result['path']
        
        if path:
            print(f"   {from_name} → {to_name}: {path}")
        else:
            print(f"   {from_name} → {to_name}: No path found")
    except sqlite3.Error as e:
        print(f"   {from_name} → {to_name}: Error - {e}")

## 6. Cypher Query Testing

In [ ]:
# Test Cypher query parsing and validation
print("🔍 Testing Cypher Query Support:")
print()

cypher_queries = [
    "RETURN 42",
    "RETURN 'Hello, Graph!'",
    "MATCH (n) RETURN n",
    "MATCH (n:Person) RETURN n.name",
    "CREATE (n:Person {name: 'Eve'})",
    "MATCH (a)-[r:FRIENDS]->(b) RETURN a.name, b.name",
    "RETURN 2 + 3 * 4",
    "RETURN true AND false"
]

for query in cypher_queries:
    try:
        cursor.execute("SELECT cypher_parse(?) as result", (query,))
        result = cursor.fetchone()
        print(f"   ✅ '{query}'")
    except sqlite3.Error as e:
        print(f"   ❌ '{query}' - {e}")

## 7. Write Operations (Advanced)

In [ ]:
# Test Cypher write operations
print("✍️  Testing Write Operations:")
print()

try:
    # Begin a write transaction
    cursor.execute("SELECT cypher_begin_write() as result")
    result = cursor.fetchone()
    print(f"✅ Transaction started: {result['result']}")
    
    # Create a new node
    cursor.execute("SELECT cypher_create_node(?, ?, ?) as node_id",
                   (None, "Person", '{"name": "Eve", "age": 27, "city": "Portland"}'))
    result = cursor.fetchone()
    eve_id = result['node_id']
    print(f"✅ Created Eve with ID: {eve_id}")
    
    # Set a property
    cursor.execute("SELECT cypher_set_property(?, ?, ?, ?) as result",
                   ("node", eve_id, "occupation", "Researcher"))
    result = cursor.fetchone()
    print(f"✅ Set occupation for Eve: {result['result']}")
    
    # Commit the transaction
    cursor.execute("SELECT cypher_commit_write() as result")
    result = cursor.fetchone()
    print(f"✅ Transaction committed: {result['result']}")
    
except sqlite3.Error as e:
    print(f"❌ Write operation error: {e}")

# Check final node count
cursor.execute("SELECT graph_count_nodes() as count")
result = cursor.fetchone()
print(f"\nFinal node count: {result['count']}")

## 8. Building a Larger Graph

In [ ]:
# Create a larger social network for demonstration
import random

print("🏗️  Building a larger social network...")
print()

# Add more people
additional_people = [
    (10, {"name": "Frank", "age": 35, "occupation": "Teacher", "city": "Boston"}),
    (11, {"name": "Grace", "age": 29, "occupation": "Artist", "city": "Seattle"}),
    (12, {"name": "Henry", "age": 31, "occupation": "Engineer", "city": "Denver"}),
    (13, {"name": "Iris", "age": 26, "occupation": "Writer", "city": "Portland"})
]

for person_id, properties in additional_people:
    cursor.execute("SELECT graph_node_add(?, ?) as result",
                   (person_id, json.dumps(properties)))
    result = cursor.fetchone()
    print(f"   ✅ Added {properties['name']} (ID: {person_id})")

# Create random connections
relationship_types = ["FRIENDS", "COLLEAGUES", "KNOWS"]
new_connections = [
    (1, 10, "FRIENDS", {"since": "2021"}),
    (2, 11, "COLLEAGUES", {"department": "Creative"}),
    (3, 12, "KNOWS", {"met_at": "Conference"}),
    (4, 13, "FRIENDS", {"since": "2020"}),
    (10, 11, "FRIENDS", {"since": "2019"}),
    (12, 13, "COLLEAGUES", {"project": "Innovation"})
]

print("\n   Creating new connections...")
for from_id, to_id, rel_type, properties in new_connections:
    cursor.execute("SELECT graph_edge_add(?, ?, ?, ?) as result",
                   (from_id, to_id, rel_type, json.dumps(properties)))
    print(f"   ✅ Connected {from_id} --{rel_type}--> {to_id}")

# Final statistics
cursor.execute("SELECT graph_count_nodes() as nodes, graph_count_edges() as edges")
final_stats = cursor.fetchone()
print(f"\n📊 Final Network Statistics:")
print(f"   Nodes: {final_stats['nodes']}")
print(f"   Edges: {final_stats['edges']}")

try:
    cursor.execute("SELECT graph_density() as density")
    result = cursor.fetchone()
    print(f"   Density: {result['density']:.3f}")
except sqlite3.Error as e:
    print(f"   Density: {e}")

## 9. Cleanup and Summary

In [ ]:
# Summary of what we accomplished
print("🎉 Tutorial Summary:")
print("=" * 50)
print("✅ Loaded SQLite Graph Extension")
print("✅ Created nodes with JSON properties")
print("✅ Created edges with relationships and properties")
print("✅ Calculated graph statistics (density, connectivity)")
print("✅ Analyzed node centrality")
print("✅ Found shortest paths between nodes")
print("✅ Tested Cypher query parsing")
print("✅ Performed write operations")
print("✅ Built a larger social network graph")

print("\n💡 Next Steps:")
print("   - Try building your own graph data")
print("   - Experiment with different algorithms")
print("   - Explore more complex Cypher queries")
print("   - Test performance with larger datasets")
print("   - Integrate with your applications")

# Close connection
conn.close()
print("\n🔚 Database connection closed.")